In [21]:
## Retrieval Augmentation Generation 
import os
from dotenv import load_dotenv
load_dotenv()

True

In [22]:
os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")

In [23]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
documents=SimpleDirectoryReader("data").load_data()

In [24]:
documents

[Document(id_='6a7a6a3a-a2df-4f84-8a14-55c1d8ab3b49', embedding=None, metadata={'page_label': '1', 'file_name': 'gpt4-0.pdf', 'file_path': 'c:\\Users\\patel\\Desktop\\projects\\RAG-llama\\data\\gpt4-0.pdf', 'file_type': 'application/pdf', 'file_size': 3100, 'creation_date': '2024-05-14', 'last_modified_date': '2024-05-14'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text="OpenAI is launching GPT-4o, an iteration of the GPT-4 model that powers its hallmark product, ChatGPT.\nThe updated model ªis much fasterº and improves ªcapabilities across text, vision, and audio,º OpenAI \nCTO Mira Murati said in a livestream announcement on Monday. It'll be free for all users, and paid users \nwill continue to ªhave up to five times the capacity limitsº of free users,

In [25]:
import openai
openai.api_key = "sk-proj-TFOi0qPst9Lkzh4L36jnT3BlbkFJd4ukE2DVE7Rz26AbfdyQ"
index=VectorStoreIndex.from_documents(documents,show_progress=True)

Parsing nodes:   0%|          | 0/6 [00:00<?, ?it/s]

Generating embeddings: 100%|██████████| 6/6 [00:00<00:00, 16.31it/s]


In [26]:
index

In [27]:
query_engine=index.as_query_engine()

In [28]:
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.indices.postprocessor import SimilarityPostprocessor
# Retrieving 4 outputs and specifying the condition that show response > similarity of 0.80
retriever=VectorIndexRetriever(index=index,similarity_top_k=4)
postprocessor=SimilarityPostprocessor(similarity_cutoff=0.80)

query_engine=RetrieverQueryEngine(retriever=retriever,
                                  node_postprocessors=[postprocessor])

In [29]:
query_engine

In [30]:
response = query_engine.query("What is the earnings of JP Morgan")

In [31]:
response

Response(response='The earnings of JPMorgan Chase & Co. can be determined by looking at the overall gain percentages provided in the context.', source_nodes=[NodeWithScore(node=TextNode(id_='1f29e8b9-006b-435d-a954-7f840a28bc0f', embedding=None, metadata={'page_label': '2', 'file_name': 'jpmorgan_annualreport-2022-splitted.pdf', 'file_path': 'c:\\Users\\patel\\Desktop\\projects\\RAG-llama\\data\\jpmorgan_annualreport-2022-splitted.pdf', 'file_type': 'application/pdf', 'file_size': 103636, 'creation_date': '2024-05-14', 'last_modified_date': '2024-05-13'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='a1f22fea-19f3-4fc8-9092-8361b4837b86', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'page_label': '2', 

In [32]:
print(response)

The earnings of JPMorgan Chase & Co. can be determined by looking at the overall gain percentages provided in the context.


In [33]:
from llama_index.core.response.pprint_utils import pprint_response
pprint_response(response,show_source=True)

Final Response: The earnings of JPMorgan Chase & Co. can be determined
by looking at the overall gain percentages provided in the context.
______________________________________________________________________
Source Node 1/2
Node ID: 1f29e8b9-006b-435d-a954-7f840a28bc0f
Similarity: 0.8252268101921668
Text: Stock total return analysis Bank One S&P 500 Index S&P
Financials Index Performance since becoming CEO of Bank One
(3/27/2000—12/31/2022)1 Compounded annual gain 11.3% 6.1% 4.6% Overall
gain 1,047.8% 287.7% 176.1% JPMorgan Chase & Co. S&P 500 Index S&P
Financials Index Performance since the Bank One and JPMorgan Chase &
Co. merger (7/1/2004—12/...
______________________________________________________________________
Source Node 2/2
Node ID: ba7e0ce4-2375-437b-a31c-83b9276d4f8d
Similarity: 0.8032796989389438
Text: Stock total return analysis Bank One S&P 500 Index S&P
Financials Index Performance since becoming CEO of Bank One
(3/27/2000—12/31/2022)1 Compounded annual gain 11.3% 6.1

In [34]:

import os.path
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    StorageContext,
    load_index_from_storage,
)

# check if storage already exists
PERSIST_DIR = "./storage"
if not os.path.exists(PERSIST_DIR):
    # load the documents and create the index
    documents = SimpleDirectoryReader("data").load_data()
    index = VectorStoreIndex.from_documents(documents)
    # store it for later
    index.storage_context.persist(persist_dir=PERSIST_DIR)
else:
    # load the existing index
    storage_context = StorageContext.from_defaults(persist_dir=PERSIST_DIR)
    index = load_index_from_storage(storage_context)

# either way we can now query the index
query_engine = index.as_query_engine()
test1 = query_engine.query("Who built openai?")
response = query_engine.query("how was the performance of JPMorgan ?")
test = query_engine.query("What is openai voice assistant?")
test2 = query_engine.query("How did future losses changed dramatically?")
test3 = query_engine.query("How much are the reserves?")

print(response)
print(test)
print(test1)
print(test2)
print(test3)

JPMorgan's performance since the merger with Bank One showed a compounded annual gain of 9.9% and an overall gain of 471.6% from July 1, 2004, to December 31, 2022. Additionally, the performance for the period ending December 31, 2022, indicated a one-year compounded annual loss of 12.6%, a five-year gain of 7.7%, and a ten-year gain of 14.9%.
OpenAI Voice Assistant is a conversational AI system developed by OpenAI that is designed to interact with users through voice commands and responses. It uses advanced natural language processing and machine learning techniques to understand and generate human-like speech, enabling users to engage in dialogue and perform tasks using voice interactions.
OpenAI was founded by Elon Musk, Sam Altman, Greg Brockman, Ilya Sutskever, John Schulman, and Wojciech Zaremba.
The future losses changed dramatically due to the scenarios used to estimate future credit losses undergoing significant shifts.
$16 billion


In [35]:
test4 = query_engine.query("How will developers access GPT-4o?")

In [36]:
print(test4)

Developers will likely access GPT-4o through an API provided by the organization or company that develops and maintains GPT-4o. This API will allow developers to integrate GPT-4o's capabilities into their applications, platforms, or systems, enabling them to leverage the power of GPT-4o for various natural language processing tasks.


In [37]:
test5= query_engine.query("What was the Net Income in 2020, the Reported one?")
print(test5)

$29.1 billion


In [38]:
test6 = query_engine.query("What was the evolution of the Net Income from 2020? just Net Income from 2020?")
print(test6)

$29.1 billion


In [39]:
test6 = query_engine.query("What was the evolution in term of percentage of the Net Income from 2020 to 2021, the Reported one?")

print(test6)

The evolution in terms of percentage of the Reported Net Income from 2020 to 2021 was a decrease from 14.4% in 2020 to 12.09% in 2021.
